# Genetic Algorithms
## Evolutionary Algorithms for Optimization

### Mathematical Optimization
Optimization is the process of finding some numerical values that generate a minimum or maximum value for a specified function. Examples include finding the best parameters for a statistical distribution fit to some data, numerically solving differential equations, or feature selection in machine learning.

There are many types and classes of optimization algorithms. Most that are invented by mathematicianas and computer scientists rely on function derivatives / gradients. Anybody who's studied the topic even slightly will likely remember [Newton-Raphson](https://en.wikipedia.org/wiki/Newton%27s_method) or the [BFGS](https://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm) - the latter of which even need the second derivaties (the [Hessian](https://en.wikipedia.org/wiki/Hessian_matrix)). Mathematical optimization algorithms typically iterate over two steps:

1.  evaluate current solution
2. find new solution to try

A series of individual solutions are identified and evluated, until the sequence converges to a solution.

Gradient-following methods can have good properties, but also two major issues.

1. The first is the need to compute derivatives. This can often be very difficult - even assuming derivatives can be analytically solved - and time consuming. Furthermore, not all functions we wish to optimizte even have a derivative. What is the derivative of an accuracy loss function for a [Decision Tree Classifier](https://en.wikipedia.org/wiki/Decision_tree_learning)?

2. The second issue is that gradient-followers can easily get stuck in local optima, rather than finding a global optimum point. Most such algorithms rely on being provided an initial solution, at which point the necessary derivatives are computed to determine the direction (and perhaps distance) to find the next point to evaluate. Depending on the curvature of the function to optimize, and the initial solution, the algorithm may get stuck in a local, not global, optimum.

In [1]:
# TOOO: demo finding local not global optimum

### Non-Gradent Following Optimization Algorithms

There are several types of mathematical optimization algorithms which similarly operate on a sequence of individual solutions, but don't use derivatives. I am familiar with [Simulated Annealing](https://en.wikipedia.org/wiki/Simulated_annealing) and [Golden Section Search](https://en.wikipedia.org/wiki/Golden-section_search) opimizatin.

### Evolutionary Algorithms
The term [Evolutionary Algorithm (EA)](https://en.wikipedia.org/wiki/Evolutionary_algorithm) refers to a population-based metaheuristic optimization algorithm, and is a subset of evolutionary computation. Broadly, evolutionary algorithms are designed around concepts which come from biological evolution. There are several classes of evolutionary algorithms:

- Differential Evolution
- Evolutionary Programming
- Evolutionary Strategy
- Genetic Algorithm
- Genetic Programming ([see here](https://github.com/ahowe42/baseball))
- Learning Classifier System
- Neuroevolution

This set of lectures is focused on the Genetic Algorithm (GA), but could potentially be extended to include [Genetic Programming](https://en.wikipedia.org/wiki/Genetic_programming).

### Other Gradient Eschewing Algorithms
In addition to EA's, there are several other types of metaheuristic optimization algorithms that are based on the idea of optimizing with a population of potential solutions. These include:

- [Ant Colony Optimization - or Traveling Ant Colony Optimization (TACO)](https://en.wikipedia.org/wiki/Ant_colony_optimization_algorithms)
- [Particle Swarm Optimization](https://en.wikipedia.org/wiki/Particle_swarm_optimization)
- [Bees Algorithm](https://en.wikipedia.org/wiki/Bees_algorithm)
- [Adaptive Dimensional Search](https://en.wikipedia.org/wiki/Adaptive_dimensional_search)
- [Gaussian Adaptation](https://en.wikipedia.org/wiki/Gaussian_adaptation)
- Harmony Search - special case of [Evolution Strategy](https://en.wikipedia.org/wiki/Evolution_strategy)

I am only familiar with a few of these.